# numpyだけで実装するアルファベット26文字画像認識
<a5>フレームワークを一切使用せずnumpyで画像認識ニューラルネットワークを実装<br></a5>

In [1]:
"""
必要なライブラリをインポート
"""
# 公開ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from PIL import Image
from sklearn.model_selection import train_test_split

# 自作ライブラリ
from libs.layers import TwoConvNet7
from libs.optimizer import RMSProp, Adam, Adadelta

Using TensorFlow backend.


# すべてのテストデータで検証
<a5>学習時の損失と精度は500枚の画像で算出したので、テストデータすべての場合を確認する</a5>

In [6]:
"""
全てのテストデータで検証
"""
# emnist 読み込み
# 今回はletters（アルファベット小文字a~z）を対象
from emnist import extract_training_samples, extract_test_samples
x_train, y_train = extract_training_samples("letters")
x_test, y_test = extract_test_samples('letters')

# 画像の前処理
img_side = 28
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

#  labelsのOne-Hotベクトル化
y_train = y_train - 1;
y_test = y_test - 1;
print(y_test.tolist()[:10])
print(y_train.tolist()[:15])
y_test = np.identity(num_classes)[y_test]
y_train = np.identity(num_classes)[y_train]

# 白黒反転＋正規化
X_train = -(x_train - 255) /255
X_test = -(x_test - 255) / 255
X_train = X_train.reshape(-1,1,28,28)
X_test = X_test.reshape(-1,1,28,28)
print(X_test.shape)

with open("model.pickle", "rb") as f:
    model = pickle.load(f)
    
#　すべてのテストデータで精度算出
total_acc = 0
total_loss = 0
alldata_acc = 0
alldata_loss = 0
val_batch_size = 500
val_iter_num = X_test.shape[0] // (val_batch_size) - 1
print(val_iter_num)
for it in range(val_iter_num):
    accuracy = model.accuracy(X_test[val_batch_size * it:val_batch_size*(it+1)], y_test[val_batch_size * it:val_batch_size*(it+1)])
    loss  = model.loss(X_test[val_batch_size * it:val_batch_size*(it+1)], y_test[val_batch_size * it:val_batch_size*(it+1)])
    total_acc += accuracy
    total_loss += loss

alldatas_acc = total_acc / val_iter_num
alldatas_loss = total_loss / val_iter_num
print("acc ; ",alldatas_acc)
print("loss: ",alldatas_loss)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[22, 6, 15, 14, 22, 16, 12, 10, 21, 23, 9, 13, 17, 20, 25]
(20800, 1, 28, 28)
40
acc ;  0.93855
loss:  0.48950054390346853


<a5>一部データに対して精度が低下：95.2% → 93.8%<br>データ数が増えた分、後者の方が実際の精度として信頼できるはず。<br>過学習まではしていないので、リソースをより増やした環境でモニタしながらハイパーパラメータを調整する必要がありそう。</a5>